In [1]:
import torch
import torch.nn as nn

In [6]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(4, 5)
        self.layer2 = nn.Linear(5, 2)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        return self.layer2(x)

# Method A: apply a custom init function
def preset_init(m):
    if isinstance(m, nn.Linear):
        # fill all weights with 0.5, all biases with -0.1
        m.weight.data.fill_(0.5)
        m.bias.data.fill_(-0.1)

model_a = SimpleMLP()
model_a.apply(preset_init)

# Now model_a’s weights are deterministic for testing:
inp = torch.ones(1, 4)

print(model_a)
print(model_a(inp))

SimpleMLP(
  (layer1): Linear(in_features=4, out_features=5, bias=True)
  (layer2): Linear(in_features=5, out_features=2, bias=True)
)
tensor([[4.6500, 4.6500]], grad_fn=<AddmmBackward0>)


In [5]:
# Method B: load_state_dict from a preset dict
model_b = SimpleMLP()
preset = {
    'layer1.weight': torch.full((5, 4), 0.2),
    'layer1.bias':   torch.zeros(5),
    'layer2.weight': torch.eye(2, 5),    # identity-like
    'layer2.bias':   torch.tensor([1.0, -1.0]),
}
model_b.load_state_dict(preset, strict=False)

# Test
print(model_b)
print(model_b(inp))
# → again, fully deterministic

SimpleMLP(
  (layer1): Linear(in_features=4, out_features=5, bias=True)
  (layer2): Linear(in_features=5, out_features=2, bias=True)
)
tensor([[ 1.8000, -0.2000]], grad_fn=<AddmmBackward0>)
